## Scoping

Nearly a billion people will go hungry tonight, yet this year the U.S. will turn nearly 5 billion bushels of corn into ethanol. That’s enough food to feed 412 million people for an entire year.

How about Canada? Are Canadian farmers lowering wheat production for canola?


# ETL (Extract, Transform, Load)

In [ ]:
!pip install -q contextily geopandas folium mapclassify

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sb
import contextily as ctx
import folium
plt.rcParams['figure.figsize'] = [12, 8]

## Yield

In [ ]:
filepath = 'D:/Users/wttz1/OneDrive/Documents/GitHub/Palette_Cohort_4/Final_Project/Data'

In [ ]:
df_sk = pd.read_csv(filepath + '/rm-yields-data.csv', header=0)

In [ ]:
df_mb = pd.read_excel(filepath + '/MMPP - Yield by Soil Type Browser.xlsx', header=0)

## Shapefiles

In [ ]:
gdf_sk=gpd.read_file(filepath + '/AB_shape/RuralMunicipality.shp')

In [ ]:
gdf_mb=gpd.read_file(filepath+'/MB_shape/MB_Municipal_Boundaries.shp')

## Transforming

In [ ]:
# List of columns
df_mb.columns

In [ ]:
# Info about columns
df_mb.info()

In [ ]:
# Table heads
df_mb.head()

In [ ]:
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].str.replace(' Tonnes', '') # Replacing Tonnes
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].replace('Tolerance', np.NaN) # Replacing Tolerance
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].astype(float) # changing object to float data type
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('RM OF ','')
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('RM OF ','')
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('MUNICIPALITY OF ','', regex=False) # not caps sensitive\n",
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace(' MUNICIPALITY','', regex=False) # not caps sensitive\n",
df_mb['Risk Area / R.M.'] = df_mb['Risk Area / R.M.'].str.replace('CITY OF ','', regex=False) # not caps sensitive"

In [ ]:
df_mb_pivot=pd.pivot_table(df_mb.drop(columns=['Yield/acre(Metric).1', 'Yield/acre(Imperial)', 'Soil', 'Farms' ]),
               index=['Risk Area / R.M.', 'Year'], columns='Crop', values='Yield/acre(Metric)')\
               .reset_index()

In [ ]:
df_mb_pivot.columns

In [ ]:
df_mb_clean = df_mb_pivot.rename(columns={
    'Risk Area / R.M.': 'RM',
    'ARGENTINE CANOLA': 'Canola',
    'BARLEY':'Barley',
    'CANARYSEED': 'Canary Seed',
    'DURUM WHEAT': 'Durum Wheat',
    'LENTILS': 'Lentils',
    'OATS': 'Oats',
    'RED SPRING WHEAT': 'Spring Wheat',
    'WHITE PEA BEANS': 'Peas'}) \
        .drop(columns=['ALFALFA', 'FABABEANS', 'FABABEANS', 'POLISH CANOLA'] )

In [ ]:
# Crop conversion in MB tonnes to bushel
df_mb_clean['Canola']=df_mb_clean['Canola'] * 44.092
df_mb_clean['Barley']=df_mb_clean['Barley'] * 45.93
df_mb_clean['Canary Seed']=df_mb_clean['Canary Seed'] * 44.092
df_mb_clean['Durum Wheat']=df_mb_clean['Durum Wheat'] * 36.74
df_mb_clean['Lentils']=df_mb_clean['Lentils'] * 36.74
df_mb_clean['Oats']=df_mb_clean['Oats'] * 64.842
df_mb_clean['Spring Wheat']=df_mb_clean['Spring Wheat'] * 36.74
df_mb_clean['Peas']=df_mb_clean['Peas'] *  36.74

In [ ]:
# Creating province column
df_mb_clean['Province'] = 'MB'

In [ ]:
df_mb_clean.head()

In [ ]:
df_sk.columns

In [ ]:
df_sk.info()

In [ ]:
df_sk.head()

In [ ]:
df_sk_clean=df_sk.rename(columns={'Durum': 'Durum Wheat'}).drop(columns=['Winter Wheat', 'Mustard', 'Sunflowers', 'Fall Rye', 'Spring Rye', 'Tame Hay','Flax', 'Chickpeas' ] )

In [ ]:
# Crop Conversion in SK - pounds to bushels
df_sk_clean['Lentils']=df_sk_clean['Lentils'] / 60
df_sk_clean['Canary Seed']=df_sk_clean['Canary Seed'] / 50

In [ ]:
# Creating province column
df_sk_clean['Province'] = 'SK'

In [ ]:
df_sk_clean.head()

In [ ]:
df_sk_clean.describe()

In [ ]:
df_mb_clean.head()

In [ ]:
df_mb_clean.describe()

In [ ]:
# More like copy paste in excel under existing data. Make sure column names are the same
df = pd.concat([df_mb_clean, df_sk_clean])

In [ ]:
df = df[df['Year']>=2003] ## working on 2003 to 2022

In [ ]:
wheat_null_count = df['Spring Wheat'].isna().sum()
print('Number of null values in Spring Wheat:', wheat_null_count)

wheat_total_count = len(df['Spring Wheat'])
print('Total number of values in Spring Wheat:', wheat_total_count)

# EDA(Exploratory Data Analysis)

## Missing Values

In [ ]:
df.loc[df['Year']>=2003].isna().sum().sort_values().plot(kind='bar', color='green')
plt.title('Missing Values - 2003 to 2022')
plt.xlabel('Crops')
plt.ylabel('# of Missing values')

plt.xticks(rotation=45)

plt.axhline(len(df)/2, linestyle='--', color='red')
plt.axhline(len(df)/4, linestyle='--', color='red')
plt.axhline(len(df)/10, linestyle='--', color='red')
plt.show()

## Outliers

In [ ]:
df = df[df['Year']>=2003]
df = df.dropna(subset=['Spring Wheat'])
crop_columns=['Canola', 'Barley', 'Canary Seed', 'Durum Wheat',
       'Lentils', 'Oats', 'Spring Wheat', 'Peas']

## Histograms

In [ ]:
df['Spring Wheat'].hist(bins=300)
plt.show()

In [ ]:
sb.heatmap(df[crop_columns].corr(), annot=True)

## Boxplots

In [ ]:
df[crop_columns].boxplot()

In [ ]:
df.loc[df['Spring Wheat']>180]

In [ ]:
# replace the 2018 wheat data with the average of 2017 and 2019
average_value = df[(df['RM'] == 314) & ((df['Year'] == 2017) | (df['Year'] == 2019))]['Spring Wheat'].mean()
condition = (df['RM'] == 314) & (df['Year'] == 2018)
df.loc[condition, 'Spring Wheat'] = average_value

In [ ]:
df['Spring Wheat'].hist(bins=300)
plt.show()

In [ ]:
# Peorson Corr (-1 to 1), -1 negative corr, o no corr, 1 positive corr
# Using Seaborn
sb.heatmap(df[crop_columns].corr(),annot=True, cmap='Greens')


## GIS Analyis

In [ ]:
# CRS --> Cordinate Reference Systems
gdf_mb['geometry'].crs

In [ ]:
gdf_mb

In [ ]:
# Standardazing CRS formats
gdf_sk['geometry']=gdf_sk['geometry'].to_crs('epsg:4326')
gdf_mb['geometry']=gdf_mb['geometry'].to_crs('epsg:4326')

In [ ]:
gdf_sk.explore()

In [ ]:
gdf_mb.explore()

In [ ]:
# Renaming column name
gdf_sk['RMNO']=gdf_sk['RMNO'].astype(int)

In [ ]:
gdf=pd.concat([gdf_sk[['RMNO','geometry']].rename(columns={'RMNO':'RM'}),gdf_mb[['MUNI_NAME', 'geometry']].rename(columns={'MUNI_NAME':'RM'})])
gdf

In [ ]:
set(gdf["RM"].unique()) - set(df_sk_clean["RM"].unique())   

In [ ]:
gdf_mb['MUNI_NAME']=gdf_mb['MUNI_NAME'].str.replace('RM OF ','')
gdf.loc[gdf["RM"] == "ST. FRANCOIS XAVIER", "RM"] = "ST. FRANCIS XAVIER"
gdf.loc[gdf["RM"] == "DE SALABERRY", "RM"] = "DESALABERRY"
df.loc[df["RM"] == "EAST ST PAUL", "RM"] = "EAST ST. PAUL"
df.loc[df["RM"] == "KILLARNEY-TURTLE MTN" "RM"] = "KILLARNEY-TURTLE MOUNTAIN"
gdf.loc[gdf["RM"] == "ROBLIN", "RM"] = "HILLSBURG-ROBLIN-SHELL RIVER"

In [ ]:
gdf=pd.concat([gdf_sk[['RMNO','geometry']].rename(columns={'RMNO':'RM'}),gdf_mb[['MUNI_NAME', 'geometry']].rename(columns={'MUNI_NAME':'RM'})])
gdf

In [ ]:
gdf.plot() #not interactive
#gdf.explore()

In [ ]:
gdf['RM']=gdf['RM'].astype('string')
df['RM']=df['RM'].astype('string')

In [ ]:
gdf = gdf.reset_index()

In [ ]:
df

In [ ]:
# SQL inner join
df_gdf=pd.merge(gdf, df, on='RM', how='inner')

In [ ]:
df_gdf

In [ ]:
print('RM Before merging', gdf['RM'].nunique())
print('RM After merging',df['RM'].nunique())
print('RM After merging',df_gdf['RM'].nunique())

In [ ]:
ds=df_gdf.copy()

In [ ]:
stats = ds.groupby (["RM"]).agg({"Spring Wheat":["mean","std","min","max"]})
stats.to_csv (filepath + '/wheat_stats.csv')
stats.columns = ['_'.join(col) for col in stats.columns.values]
print(stats.columns)
ds.loc[ds["Province"] == "MB"]    # SK or MB
print (stats)

In [ ]:
# Replace nan with 0
ds['Spring Wheat'] = ds['Spring Wheat'].fillna(0)

### GIS Visualization

#### Explore functiom

In [ ]:
ds

In [ ]:
m = folium.Map(location=[-37.5, 143.5], zoom_start=9)
ax = ds.loc[ds['Year'] >= 2003].plot(column='Spring Wheat', legend=True, cmap='Greens', figsize=(10, 20),scheme="quantiles")

# Add web map tiles
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite, zoom=20)

plt.show()
title_html = '''
                 <h3 align="center" style="font-size:30px; color:Green;"><b> Spring Wheat Yield 2003 - 2022 </b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))
m.save(filepath + '/Spring_Wheat.html')

#### Plot

In [ ]:
ds.plot(column='Spring Wheat', 
                                 legend=True,  
                                 cmap='Greens')

## Aggragetions

In [ ]:
df_03_22=df.loc[df['Year']>=2003].sort_values(['RM', 'Year'])

In [ ]:
df_03_22.to_csv(filepath +"/df_03_22.csv")

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(4, 5, figsize=(32, 18),sharey=True)
years = df_03_22['Year'].unique()

for i, year in enumerate(years):
    ax = axes[i//5, i%5]
    merged_df = pd.merge(gdf, df_03_22.loc[df_03_22['Year'] == year], on='RM')
    merged_df.plot(column='Spring Wheat', cmap='RdYlGn', legend=True, ax=ax, scheme='quantiles')
    ax.set_title(f'Canola Yield in {year}', color='Blue', size=12)

plt.tight_layout()
plt.show()

In [ ]:
df_agg = stats
df_agg = df_agg.dropna()

In [ ]:
df_agg

In [ ]:
df_agg.info()

## K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans 

# Select the columns with the correct MultiIndex format
df_agg_can = df_agg[['Spring Wheat_mean', 'Spring Wheat_std']]

# Let's define our features
X = df_agg_can.copy()

from sklearn.metrics import silhouette_score
n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]  # number of clusters
clusters_inertia = []  # inertia of clusters
s_scores = []  # silhouette scores

for n in n_clusters:
    KM = KMeans(n_clusters=n, init='k-means++', n_init=10).fit(X)
    clusters_inertia.append(KM.inertia_)  # data for the elbow method
    silhouette_avg = silhouette_score(X, KM.labels_)
    s_scores.append(silhouette_avg)  # data for the silhouette score method


## Elbow Metod

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(n_clusters, clusters_inertia, 'o-', color='blue', label='Elbow Method')
ax.set_title("Elbow Method")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Clusters Inertia")
ax.axvline(4, ls="--", c="red")
ax.axvline(5, ls="--", c="red")
ax.axvline(6, ls="--", c="red")
plt.grid()
plt.legend()
plt.show()

## Silhouette Score

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(n_clusters, s_scores, 's-', color='green', label='Silhouette Score Method')
ax.set_title("Silhouette Score Method")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Silhouette Score")
ax.axvline(4, ls="--", c="red")
plt.grid()
plt.legend()
plt.show()


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, init='k-means++', random_state=42, n_init=10)
df_agg_can['Clusters_4']=kmeans.fit_predict(df_agg_can[['Spring Wheat_mean', 'Spring Wheat_std']])
df_agg_can.loc[:, 'Clusters_4'] = df_agg_can['Clusters_4'].astype(int)

In [ ]:
sb.scatterplot(data=df_agg_can, x='Spring Wheat_mean', y='Spring Wheat_std', hue='Clusters_4')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 100, c = 'blue', label = 'Centroids')
plt.title('Spring Wheat Clustering Mean and Std | 2003-2022 | K-Means ', color='blue', size =14)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42, n_init=10)
df_agg_can['Clusters_5']=kmeans.fit_predict(df_agg_can[['Spring Wheat_mean', 'Spring Wheat_std']])
df_agg_can.loc[:, 'Clusters_5'] = df_agg_can['Clusters_5'].astype(int)

In [ ]:
sb.scatterplot(data=df_agg_can, x='Spring Wheat_mean', y='Spring Wheat_std', hue='Clusters_5')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 100, c = 'blue', label = 'Centroids')
plt.title('Spring Wheat Clustering Mean and Std | 2003-2022 | K-Means ', color='blue', size =14)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=6, init='k-means++', random_state=42, n_init=10)
df_agg_can['Clusters_6']=kmeans.fit_predict(df_agg_can[['Spring Wheat_mean', 'Spring Wheat_std']])
df_agg_can.loc[:, 'Clusters_6'] = df_agg_can['Clusters_6'].astype(int)

In [ ]:
sb.scatterplot(data=df_agg_can, x='Spring Wheat_mean', y='Spring Wheat_std', hue='Clusters_6')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 100, c = 'blue', label = 'Centroids')
plt.title('Spring Wheat Clustering Mean and Std | 2003-2022 | K-Means ', color='blue', size =14)
plt.show()

In [ ]:
df_agg_can

In [ ]:
df_agg_can.to_csv(filepath +"/sg_agg_can.csv")

In [ ]:
pd.merge(
    gdf,
    df_agg_can,
    on='RM'
).explore(column='Clusters_4', legend='True', k=4, scheme='naturalbreaks', cmap='Oranges')

## Ranking clusters based on Mean

In [ ]:
df_agg_can.groupby('Clusters_4').mean()\
    .sort_values('Spring Wheat_mean')[['Spring Wheat_mean',	'Spring Wheat_std']]

In [ ]:
df_agg_can.groupby('Clusters_5').mean()\
    .sort_values('Spring Wheat_mean')[['Spring Wheat_mean',	'Spring Wheat_std']]

In [ ]:
df_agg_can.groupby('Clusters_6').mean()\
    .sort_values('Spring Wheat_mean')[['Spring Wheat_mean',	'Spring Wheat_std']]

In [ ]:
# Ranking based on the mean
df_agg_can['Clusters_4_ranked']=df_agg_can['Clusters_4'].replace(to_replace={
    0:1,
    2:2,
    1:3,
    3:4
})

In [ ]:
df_agg_can.groupby('Clusters_4').mean()\
    .sort_values('Spring Wheat_std')[['Spring Wheat_mean',	'Spring Wheat_std']]

In [ ]:
# Ranking based on the STD or volatility
df_agg_can['Clusters_4_ranked_std']=df_agg_can['Clusters_4'].replace(to_replace={
    0:1,
    2:2,
    1:3,
    3:4
})

In [ ]:
pd.merge(
    gdf,
    df_agg_can,
    on='RM'
).explore(column='Clusters_4_ranked', legend='True', k=4, scheme='naturalbreaks', cmap='Oranges')